In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

In [2]:
# Import OpenAI chat model
from langchain.chat_models import ChatOpenAI
import os

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Create the OpenAI chat model
chat = ChatOpenAI(
    model="gpt-4",  # Use "gpt-3.5-turbo" if you want a cheaper alternative
    temperature=0.2
)

/var/folders/wb/j4k9h7qx311cp9pg23ddgxdc0000gp/T/ipykernel_30598/4040255664.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(


Tool 1: web search

In [3]:
from langchain_community.tools import TavilySearchResults, DuckDuckGoSearchResults
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import Tool

os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['SERPAPI_API_KEY'] = SERPAPI_API_KEY

# internet_search = TavilySearchResults(
#     max_results=10,
#     include_answer=True,
#     include_raw_content=True,
#     include_images=False
#     )
# internet_search = DuckDuckGoSearchResults(max_results=10)

# Initialize the SerpAPI search tool
serp_api = SerpAPIWrapper()

# Wrap it as a Tool to add name & description
internet_search = Tool(
    name="internet_search",
    description="Returns a list of relevant document snippets for a textual query retrieved from the internet.",
    func=serp_api.run  # SerpAPIWrapper does not support direct execution, so use `.run`
)

# internet_search.name = "internet_search"
# internet_search.description = "Returns a list of relevant document snippets for a textual query retrieved from the internet."


In [4]:
# internet_search.invoke({'query': 'Create a plot of the monthly average of gold price in swiss franc by gramm for the last 10 years'})

Tool 2: plot

In [5]:
import matplotlib

from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Executes python code and returns the result. The code runs in a static sandbox without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

# from langchain_core.pydantic_v1 import BaseModel, Field
# class ToolInput(BaseModel):
#     code: str = Field(description="Python code to execute.")
# repl_tool.args_schema = ToolInput

Agent

In [6]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate


# Create the prompt template
# prompt = ChatPromptTemplate.from_template("{input}")
# Create the prompt with agent_scratchpad
prompt = ChatPromptTemplate.from_template(
    "{input}\n\nPrevious Actions:\n{agent_scratchpad}"
)

# Initialize the ReAct agent
agent = create_openai_tools_agent(
    tools=[internet_search, repl_tool],
    llm=chat,
    prompt=prompt,
)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=[internet_search, repl_tool], verbose=True)


In [7]:
try:
    result = agent_executor.invoke({
        "input":"Find the gold price history in us dollar by once for the last month day by day, transform it to numerics and plot it"
    })
except AttributeError as e:
    print(f"Error: {e}")



> Entering new AgentExecutor chain...

Invoking: `internet_search` with `gold price history in us dollar for the last month`


Gold Price is at a current level of 2648.01, down from 2651.13 last month and up from 2026.18 one year ago. This is a change of -0.12% from last month and 30.69% from one year ago.I'm sorry, but the information retrieved does not provide a day-by-day history of gold prices for the last month. It only provides the current level, the level from last month, and the level from one year ago. 

To get a day-by-day history, we would need access to a specific financial database or API that provides this information. Unfortunately, I currently don't have the capability to access such databases or APIs. 

I recommend using financial data platforms like Yahoo Finance, Google Finance, or Bloomberg, which allow you to view and download historical price data for various commodities including gold.

> Finished chain.
